# Notebook 2: The Agent Loop

The manual cycle from Notebook 1, wrapped into a `while` loop that keeps running until the model says it's done.

## Setup

In [ ]:
import json
import sys
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

sys.path.insert(0, str(Path("..").resolve()))
from tools import TOOL_SCHEMAS, TOOL_FUNCTIONS

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

SYSTEM_PROMPT = (
    "You are a helpful assistant for AI Agent Insure, a specialty insurer for "
    "AI systems, autonomous agents, and ML infrastructure. "
    "Use the available tools to answer questions accurately. "
    "Call as many tools as you need — do not guess when a tool can give you the answer."
)

## Part 1: The agent loop

In [ ]:
def run_agent(user_question: str, max_iterations: int = 10, verbose: bool = True) -> str:
    """Loop: call model → if tool_calls, execute and append → repeat until stop."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_question}
    ]

    for iteration in range(max_iterations):
        if verbose:
            print(f"--- Iteration {iteration + 1} ---")

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=TOOL_SCHEMAS,
            tool_choice="auto",
            temperature=0
        )

        choice = response.choices[0]
        finish_reason = choice.finish_reason

        if verbose:
            print(f"  finish_reason: {finish_reason}")

        messages.append(choice.message)

        if finish_reason == "stop":
            return choice.message.content

        if finish_reason == "tool_calls":
            for tool_call in choice.message.tool_calls:
                fn_name = tool_call.function.name
                fn_args = json.loads(tool_call.function.arguments)

                if verbose:
                    print(f"  → Calling tool : {fn_name}")
                    print(f"    Arguments    : {fn_args}")

                result = TOOL_FUNCTIONS[fn_name](**fn_args)

                if verbose:
                    print(f"    Result       : {result}")

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })

    return f"[Agent stopped after {max_iterations} iterations without a final answer]"

## Part 2: Single tool call — automated

In [ ]:
answer = run_agent("What does Autonomous Systems & Robotics Coverage include?")
print()
print("FINAL ANSWER:", answer)

## Part 3: Two tool calls in sequence

In [ ]:
# Model calls check_eligibility first, then get_pricing_estimate — we don't tell it to.
answer = run_agent(
    "I run a healthcare AI company. Am I eligible for coverage, and if so, "
    "what would Compliance & Regulatory Shield cost for a mid-market company?"
)
print()
print("FINAL ANSWER:", answer)

## Part 4: Print the full message history

In [ ]:
def run_agent_with_history(user_question: str, max_iterations: int = 10):
    """Same as run_agent but also returns the full message history."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": user_question}
    ]

    for _ in range(max_iterations):
        response = client.chat.completions.create(
            model=MODEL, messages=messages,
            tools=TOOL_SCHEMAS, tool_choice="auto", temperature=0
        )
        choice = response.choices[0]
        messages.append(choice.message)

        if choice.finish_reason == "stop":
            return choice.message.content, messages

        if choice.finish_reason == "tool_calls":
            for tc in choice.message.tool_calls:
                args   = json.loads(tc.function.arguments)
                result = TOOL_FUNCTIONS[tc.function.name](**args)
                messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})

    return f"[Stopped after {max_iterations} iterations]", messages


answer, history = run_agent_with_history(
    "I'm a robotics startup. What coverage do you recommend and what would it cost?"
)

print("=" * 60)
print("FULL MESSAGE HISTORY")
print("=" * 60)
for i, msg in enumerate(history):
    role = msg["role"] if isinstance(msg, dict) else msg.role

    if hasattr(msg, "tool_calls") and msg.tool_calls:
        print(f"[{i}] ASSISTANT (tool_calls):")
        for tc in msg.tool_calls:
            print(f"     → {tc.function.name}({tc.function.arguments})")
    elif isinstance(msg, dict) and msg.get("role") == "tool":
        print(f"[{i}] TOOL RESULT (id={msg['tool_call_id'][:8]}...):")
        print(f"     {msg['content']}")
    elif hasattr(msg, "content") and msg.content:
        print(f"[{i}] {msg.role.upper()}: {msg.content[:200]}")
    elif isinstance(msg, dict) and msg.get("content"):
        print(f"[{i}] {msg['role'].upper()}: {msg['content'][:200]}")
    print()

print("=" * 60)
print("FINAL ANSWER:")
print(answer)

## Part 5: Safety guard — max_iterations

In [ ]:
# max_iterations=1 forces the loop to stop early — loop runs, tool executes, no final answer.
result = run_agent(
    "What does Model & Data Security Insurance cover?",
    max_iterations=1,
    verbose=True
)
print()
print("Result:", result)

## Key concepts

- The message history is the agent's memory — every tool result is visible on the next iteration
- Always include a `max_iterations` guard
- **Next:** Notebook 3 adds `search_docs` (RAG as a tool)